# Fuzzy Logic for Python 3

In my ventures into game dev, scheduling etc. I often came across the need to normalize values and apply some logic on these values. I thought that fuzzy logic would be the best tool for this job, but the libraries I found were not easily integrated into the rest of my code and not very intuitive to use.
One of my (many) approaches was to actually build fuzzy values out of native python bools or floats, but this was not supported by the language itself due to restrictions on what can be done with bool. Then I started to collect useful math functions from different places and made them usable just by pluggin them in.

The doctests in the modules should give a good idea how to use things by themselves, while here are some examples how to use everything together.

### Functions and Sets
Defining a domain with its range and resolution should be trivial since most real world instruments come with those specifications. However, defining the fuzzy sets within those domains is where the fun begins as only a human can tell whether something is "hot" or "not", right?
I've included some simple convenience functions that combine two or more functions over a range, however I consciously left out more complex "convenience" functions such as polygons because it is probably easier to just build those as combinations of sets.

Why the distinction? Functions only map values, nothing special there at all - which is good for testing and performance. Sets on the other hand implement logical operations that have special python syntax, which makes it easy to work with but a little more difficult to test and adds some performance overhead. So, sets are for abstraction and easy handling, functions for performance.
It is possible to implement everything as pure classes as well as pure functions (been there, done that) but both approaches make things either harder to test and optimize or very hard to read.

In [ ]:
%matplotlib inline
from fuzzy.classes import Domain, Set
from fuzzy.functions import R, S, alpha

T = Domain("test", 0, 30, res=0.1)

up = Set(R(0,10))
top = Set(alpha(0.5, 1, S(0, 30)))
down = Set(S(20, 30))
T.polygon = up & top & down

T.polygon.plot()
#T.not_polygon = ~T.polygon
#T.not_polygon.plot()

### Domains

After specifying the domain and assigning sets, calling a domain with a value returns a dict of memberships of the sets in that domain.

In [ ]:
%matplotlib inline

from fuzzy.classes import Domain, Set
from fuzzy.functions import alpha, triangular
from fuzzy.hedges import plus, minus, very

numbers = Domain("numbers", 0, 20, res=0.1)

close_to_10 = Set(alpha(0.2, 0.8, triangular(0, 20)))
close_to_5 = Set(triangular(1,10))

numbers.foo = minus(close_to_5)
numbers.bar = very(close_to_10)

numbers.bar.plot()
numbers.foo.plot()
numbers.baz = numbers.foo + numbers.bar
numbers.baz.plot()

numbers(8)

### Inference

After measuring a RL value and mapping it to sets within a domain, it is normally needed to translate the result to another domain that corresponds to some sort of control mechanism. This translation or mapping is called inference and is rooted in the logical conclusion operation A => B, for example: If it rains then the street is wet.
The street may be wet for a number of reasons, but if it rains it will be wet for sure. This **IF A THEN B** can also be written as
***(A AND B) OR NOT(A AND TRUE)***. This may look straight forward for boolean logic, but since we are not just dealing with True and False, there are a number of ways in fuzzy logic to actually implement this.

In [ ]:
%matplotlib inline

from fuzzy.classes import Domain, Set, Rule
from fuzzy.functions import alpha, triangular, gauss, R, S
from fuzzy.hedges import plus, minus, very

IN = Domain("numbers in", 0, 10, res=0.1)
IN.close_5 = Set(triangular(2, 10, c=5))
OUT = Domain("numbers out", 100, 200)

In [ ]:
from numpy import outer

from fuzzy.classes import Domain, Set, Rule
from fuzzy.functions import triangular

form = Domain("form", 1, 4.1, res=1)
medium_form = Set(form, triangular(1, 4, p=3))
medium_form.plot()

diff = Domain("difficulty", 1, 5.1, res=1)
medium_difficulty = Set(diff, triangular(1,5, p=3))
medium_difficulty.plot()

# this should be
# max(min(m_a(x), m_b(x)), 1-m_a(x)) for every elem
outer([0, 0.6, 1, 0.2], [0, 0.4, 1, 0.8, 0.3, 0])


fromiter

In [7]:
%matplotlib inline

from fuzzy.classes import Domain, Set
from fuzzy.functions import alpha, triangular
from fuzzy.hedges import plus, minus, very

numbers = Domain("numbers", 0, 100, res=1e-6)

close_to_10 = Set(alpha(0.2, 0.8, triangular(0, 20)))
close_to_5 = Set(triangular(1,10))

numbers.foo = minus(close_to_5)
numbers.bar = very(close_to_10)

numbers.baz = numbers.foo + numbers.bar
numbers.foo.array()

KeyboardInterrupt: 

## Sources
* Fuzzy Logic and Control: Software and Hardware Applications, Volume 2

      By: Mohammad Jamshidi; Nader Vadiee; Timothy J. Ross - University of New Mexico
      Publisher: Prentice Hall
      Pub. Date: June 07, 1993